# LDPC Code Generation and Message Passing Decoding for Binary Symmetric and Binary Erasure Channels

### Here we create a Julia type for storing and generating the LDPC parity check matrix


In [27]:
using Distributions

#Data Structure to hold LDPC Code for quick message passing decoding
type ldpcH
    n::Int #number of variable nodes 
    k::Int 
    m::Int #number of check nodes
    mdV::Int
    mdC::Int
    dV::Array{Uint8,2}
    dC::Array{Uint8,2}
    CtoV::Array{Uint32,3}
    VtoC::Array{Uint32,3}

    #generates a structure given the parity check Matrix H
    function ldpcH(H::Array) 
        this=new()
        this.n=size(H)[2]
        this.m=size(H)[1]
        this.k=this.n-this.m

        this.dV=zeros(Uint8,this.n,1)
        this.dC=zeros(Uint8,this.m,1)
        for i=1:this.n
            this.dV[i]=sum(H[:,i])
        end
        for i=1:this.m
            this.dC[i]=sum(H[i,:])
        end
        this.mdC=maximum(this.dC)
        this.mdV=maximum(this.dV)
        this.CtoV=zeros(Uint32,this.m,this.mdC,2)
        this.VtoC=zeros(Uint32,this.n,this.mdV,2)
        #CtoV(i,cnt,1)=j jth variable node connected to check node i
        #CtoV(i,cnt,2)   such that VtoC[ CtoV[i,cnt,1] , CtoV(i,cnt,2) ,1] = i
        #vice-versa for VtoC
        
        for i=1:this.m #check node i
            count=1
            for j=1:this.n 
                if H[i,j]==1 #checking variable node j
                    this.CtoV[i,count,1]=j
                    temm=sum(H[1:i,j])
                    this.VtoC[j,temm,2]=count
                    count+=1
                end
            end
        end
        
         for j=1:this.n
            count=1
            for i=1:this.m
                if H[i,j]==1
                    this.VtoC[j,count,1]=i
                    temm=sum(H[i,1:j])
                    this.CtoV[i,temm,2]=count
                    count+=1
                end
            end
        end
        
        return this;
    end
    
    #randomly generates an LDPC code with given variable and check node degree distributions
    #does not always succeed, so might need ot be called multiple times
    #it works by randomly choosing variable node, based on the number of uninitialized edges
    #then we randomly choose a check node based on the number of uninitialized edges
    # but we make sure it does not result in a 4-cycle
    #it returns false if there are any issues, like having an even variable node degree, etc...
    
    function ldpcH(dV,dC) 
        T=new()
        T.n=size(dV)[1]
        T.m=size(dC)[1]
        T.k=T.n-T.m
        T.dV=copy(dV)
        T.dC=copy(dC)
  
        T.mdC=maximum(T.dC)
        T.mdV=maximum(T.dV)
        T.CtoV=zeros(Uint32,T.m,T.mdC,2)
        T.VtoC=zeros(Uint32,T.n,T.mdV,2)
        #CtoV(i,cnt,1)=j jth variable node connected to check node i
        #CtoV(i,cnt,2)   such that VtoC[ CtoV[i,cnt,1] , CtoV(i,cnt,2) ,1] = i
        #vice-versa for VtoC
        
        #############randomly add edges and check if they are valid##############
        cnt=0
        pv=zeros(Int64,T.n,1)
        Cc=zeros(Int64,T.m,1)
        pc=zeros(Int64,T.m,1)
        #first we come up with a probability distribution(actually more like a CDF) for the check nodes,
        pv[1]=dV[1]
        for i=2:T.n
            pv[i]=dV[i]+pv[i-1]
        end
        
        #########################################################################
        totedges=pv[T.n] #this is the total # of edges to add
        ttem=zeros(Int64,T.n,1)
        i=1
        vn=0
        
        while i <= totedges
            rnv= rand(1:pv[T.n])  #first we choose a random edge to add
            nh=true
        
            for j=1:T.n   #here we randomly choose a variable node, by going through pv
                if rnv <= pv[j] && nh
                    vn=j 
                    nh=false
                    pv[j]-=1
                elseif rnv <= pv[j] && ~nh
                    pv[j]-=1
                end
            end
       
        
            tem=0
            for j=setdiff((1:T.m),T.VtoC[vn,:,1]) #cycle through all check nodes not connected to vn to find possible check nodes
                if (T.CtoV[j,dC[j],1] == 0) #make sure the check node does not have its max # of edges 
                    pc[j]=dC[j]-Cc[j]+tem
                    tem=pc[j]
                else
                    pc[j]=tem
                end
            end
            
            if pc[T.m] <  1
                println("We were unable to find a successful checknode, err(1)")
                return false
            end
            scn=true
            while scn #here we randomly select check nodes then check if they are 'good'
                rnc= rand(1:pc[T.m])
                j=1
                while rnc > pc[j]
                    j+=1
                end
                cn=j #now we have choosen a check node and must check if it is gives girth > 4
                gcn=true
                j=1
                cnt=1
                
                if T.VtoC[vn,1] != 0 #if vn is connected to any check nodes, we must check whether adding cn will introduce a 4-cycle
                    while j <= dV[vn]
                    
                        if T.VtoC[vn,j] == 0
                            cnt=j
                            j=dV[vn]+1
                        elseif length(setdiff(intersect(T.CtoV[cn,:,1], T.CtoV[T.VtoC[vn,j],:,1]  ),0))>0 #convoluted but works for now...
                            gcn=false
                            j=dV[vn]+1
                        else
                            j+=1
                        end
                    end 
                end
                
                
                
                if gcn==true #i.e. adding cn did not result in a 4-cycle
                    T.VtoC[vn,cnt,1]=cn
                    
                    Cc[cn]+=1
                    T.VtoC[vn,cnt,2]=Cc[cn]
                    T.CtoV[cn,Cc[cn],1]=vn
                    T.CtoV[cn,Cc[cn],2]=cnt
                    scn=false
                else
                    if cn==1
                        tem=pc[1]
                    else
                        tem=pc[cn]-pc[cn-1]
                    end
                    for j=cn:T.m
                        pc[j]-=tem
                    end
                    if pc[m] <  1
                        println("We were unable to find a successful check node resulting in a girth >4 code")
                        return false
                    end
                
                end
            
            end      
            i+=1
        end
        ##########################################################################
        return T;
    end

    
    #initialize LDPC structure using alist file format
    #http://www.inference.phy.cam.ac.uk/mackay/codes/alist.html
    #note: this does not guarantee that the rows are linearly independent.
    
    function ldpcH(fname::ASCIIString) 
        f=open(fname)
        a=readline(f)
        lar=readdlm(IOBuffer(a),Int)
        T=new()
        T.n=lar[1]
        T.m=m=lar[2]
        T.k=T.n-T.m
        
        T.dV=zeros(Uint8,T.n,1)
        T.dC=zeros(Uint8,T.m,1)
        
        a=readline(f)
        lar=readdlm(IOBuffer(a),Int)
        T.mdC=maximum(lar[2])
        T.mdV=maximum(lar[1])
        
        
        T.CtoV=zeros(Uint32,T.m,T.mdC,2)
        T.VtoC=zeros(Uint32,T.n,T.mdV,2)
        
        
        a=readline(f)
        lar=readdlm(IOBuffer(a),Int)
        for i=1:T.n
            T.dV[i]=lar[i]
        end
        
        
        a=readline(f)
        lar=readdlm(IOBuffer(a),Int)
        for i=1:T.m
            T.dC[i]=lar[i]
        end
        
        
        chkcnts=zeros(Int64,T.m,1) #keeps track of number of variable nodes connected in loop
        for i=1:T.n #references variable nodes
            a=readline(f)
            lar=readdlm(IOBuffer(a),Int)
            ns=length(lar)
            for j=1:ns #cycle through all connected check nodes
                
                chkcnts[lar[j]]+=1
                
                T.VtoC[i,j,1]=lar[j]
                T.VtoC[i,j,2]=chkcnts[lar[j]]
                
                
                T.CtoV[lar[j],chkcnts[lar[j]],1]=i
                T.CtoV[lar[j],chkcnts[lar[j]],2]=j
                   
            end
        
        end
        
        close(f)
        return T
    end
    
end


function ell(dv,dc,n)
    return (log(n)-log((dv*dc-dv-dc)/(2.0*dc)))/log((dc-1.0)*(dv-1.0))
    
end
function calc_H(LH::ldpcH)
    H=zeros(Uint8, LH.m,LH.n)
    for i=1:LH.n
        for j=1:LH.dV[i]
            H[LH.VtoC[i,j,1],i]=1 
            
        end
        
    end
    return H
end
    


function  calc_G(H)
    #H is a mxn Parity check matrix
    #returns the generator Matrix G which is nxk, k=n-m
    m=size(H)[1]
    n=size(H)[2]
    k=n-m
    G=zeros(Int64,n,k)
    Hp=copy(H)
    colops=zeros(Int64,2*n,2)
    ncolo=0
    
    for i=m:-1:1#put H into systematic form , i references the current row
        findl=true
        j=i #iterates over rows
        col=n-m+i #references the column such that Hp[i,col] should eventually be the only 1 in that column
        kk=col #iterates over columns
                
        while findl #search for a column then row with a 1 in the ith position
            
            if Hp[j,kk] == 1 
                findl=false
                if kk != col
                    ncolo+=1
                    colops[ncolo,1]=kk
                    colops[ncolo,2]=col 
                    for ii=1:m
                        tem=Hp[ii,kk]
                        Hp[ii,kk]=Hp[ii,col]
                        Hp[ii,col]=tem
                    end
                    kk=col        
                end
                
                
            elseif Hp[j,kk] ==0 
                if kk>1
                    kk-=1
                elseif j>1
                    kk=col    
                    j-=1
                else
                    return false
                end              
            else
                return false
            end
        end
        #swap row vectors
        if i != j
            for ii=1:col
                tem=Hp[i,ii]
                Hp[i,ii]=Hp[j,ii]
                Hp[j,ii]=tem
            end
        end
        for ii=union(1:(i-1),(i+1):m)
            if Hp[ii,col]==1     
                for iii=1:col
                    Hp[ii,iii]=Hp[ii,iii] $ Hp[i,iii]
                end
            end
        end
       
    end
    G= [eye(Int64,k) Hp[:,1:k]' ]
    
    
    for i=ncolo:-1:1
        tem=copy(G[:,colops[i,1]])
        G[:,colops[i,1]]=copy(G[:,colops[i,2]])
        G[:,colops[i,2]]=copy(tem)
    end
   
    return G
    
    
end

function isCW(LH::ldpcH,cw::Array{Int64,2})
    n=length(cw)
    t=0
    for i=1:LH.m
        t=0
        for j=1:LH.dC[i]
            if cw[LH.CtoV[i,j,1]]==1
                t+=1
            end
        end
        if mod(t,2)==1
            return 0
        end
    end
    return 1
end

function lc_end(inp, G) #encoder for any linear code
    ni=length(inp)
    k=size(G)[1]
    n=size(G)[2]
    re=ni%k
    if re !=0 
        inp=vcat(inp,zeros(Float64,k-re,1))
        ni=length(inp)
    end
    ncw=int64(ni/k)
    cws=zeros(Float64,n*ncw,1)
    for i=1:ncw
        cws[(i-1)*n+1:n*i]=copy(inp[(i-1)*k+1:k*i]'*G %2) 
    end
    return cws
end

lc_end (generic function with 1 method)

## Encoder Type
###Here we create an encoder type, which can be initialized dependent on several criteria

In [44]:
LH=ldpcH("C:\\Users\\jtuna\\OneDrive\\Documents\\github\\LDPC\\96.33.964.alist.txt")
H=calc_H(LH)
G=calc_G(H)
i=1
j=1
for i=1:size(G)[1]
    for j=1:size(G)[2]
        @printf "%d" G[i,j]        
    end
    @printf "\n"
end
println("$i $j")
mod(H*G',2)

100000000000000000000000000000000000000000000000000000111101000001100111000010110011001111100111
010000000000000000000000000000000000000000000000010111101000011001001101111110110100000001100001
001000000000000000000000000000000000000000000000011100110101100100011111110100111101111010000100
000100000000000000000000000000000000000000000000110101100111011100100100101111101010011000101011
000010000000000000000000000000000000000000000000101110101001111111011001001010110100000011001010
000001000000000000000000000000000000000000000000010011010011000100111101011010100000100011110110
000000100000000000000000000000000000000000000000000001011110011011011111000100111001011000111010
000000010000000000000000000000000000000000000000111011001000101001111100111000110111111100010010
000000001000000000000000000000000000000000000000101101000011111000001011000000100110010000010110
000000000100000000000000000000000000000000000000000011111111001111110010010101101011000001100100
000000000010000000000000000000

48x48 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0    

# Several Different Binary Channels

In [8]:


#Binary Erasure Channel
function bec(p::Float64,cws)
    n=length(cws)
    out=zeros(Float64,n,1)
    D=rand(Bernoulli(p),n)
    for i=1:n
        if D[i]==1
            out[i]=2
        else
            out[i]=cws[i]
        end
    end
    return out
end

#Binary Symmetric Channel
function bsc(p::Float64,cws)
    n=length(cws)
    out=zeros(Float64,n,1)
    D=rand(Bernoulli(p),n)
    for i=1:n
        if D[i]==1
            out[i]=mod(cws[i]+1,2)
        else
            out[i]=cws[i]
        end
    end
    return out
end



#Binary Deletion Channel
function bdc(d::Float64,cws,dpos)
    n=length(cws)
    D=rand(Bernoulli(d),n)
    L=n-sum(D)
    out=zeros(Float64,L,1)
    j=1
    for i=1:n
        dpos[i]=D[i]
        if D[i]==0
            out[j]= cws[i]
            j+=1    
        end
    end
        
    return out
end

bdc (generic function with 1 method)

# Message Passing For the BEC and BSC Channels

In [9]:
#Message Passing for the Binary Erasure Channel
function bec_MPD(xn,LH::ldpcH,maxi)
    #xn binary vector, received message
    #p BEC probability
    #H parity check matrix
    #maxi max iterations
    n=LH.n
    m=LH.m
    
    u0=zeros(Float64,n,1) # set the llr messages
    gdcm=ones(Uint8,m,1)
    jj=0
 
    u0=copy(xn)
    tcnt=0
    msum=0
    tloc=0
    for iter=1:maxi
        #start with check messages
       
        for i=1:m #look at check node i
            tcnt=0
            msum=0
            if gdcm[i]==1
                for j=1:(LH.dC[i]) #look at all variable nodes connected to i
                    jj=LH.CtoV[i,j,1]
                    if u0[jj]==2
                        tcnt+=1
                        tloc=jj
                    else
                        msum=mod(msum+u0[jj],2)
                    end
                end
                if tcnt==0
                    gdcm[i]=0
                elseif tcnt==1
                    gdcm[i]=0
                    u0[tloc]=msum  
                end
                
            end
            
        end       
    end
    return u0
end

#message passing for the Binary Symmetric Channel
function bsc_MPD(xn,p,LH::ldpcH,maxi)
    #xn binary vector, transmitted message
    #p BSC probability
    #H parity check matrix
    #maxi max iterations
   
    n=LH.n
    m=LH.m
    
    u0=ones(Float64,n,1)*log((1.0-p)/p) # set the llr messages
    mn=zeros(Float64,n,1) #llr decoded message
    bn=zeros(Int64,n,1) #bits decode message
   
    jj=0
    tem=0.0
    temm=0
    for i=1:n
        if xn[i]==1
            u0[i]*=-1
        end
    end
    
    mVtoC=zeros(Float64,n,LH.mdV)
    mCtoV=zeros(Float64,k,LH.mdC)
    
   
    for iter=1:maxi
              
        
        #start with variable messages       
        for i=1:n #i references the variable node
            tem=u0[i]
            for j=1:LH.dV[i] #variable node i has degree dv[i]
                #jj=what is the position variable node i, in the connection to cn j
                jj=LH.VtoC[i,j,2]
                tem+=mCtoV[LH.VtoC[i,j,1] , jj ] #this is the sum of all the messages from cn's  
            end

            for j=1:LH.dV[i]
                jj=LH.VtoC[i,j,2]
                mVtoC[i,j]=tem- mCtoV[ LH.VtoC[i,j,1] , jj  ] 
            end
            
        end
        
        #now check message
        for i=1:m #look at check node i
            tem=1.0
            for j=1:(LH.dC[i]) #look at all variable nodes connected to i
                jj=LH.CtoV[i,j,2]
                tem*=tanh(mVtoC[LH.CtoV[i,j,1] , jj ]/2.0)     
            end
            for j=1:(LH.dC[i])
                jj=LH.CtoV[i,j,2]
                
                mCtoV[i,j]=2.0*atanh(tem/tanh(mVtoC[LH.CtoV[i,j,1] , jj ]/2.0) )
            end     
        end
     
        #now we attempt decoding
        for i=1:n
            mn[i]=u0[i]
            for j=1:LH.dV[i]
                cn=LH.VtoC[i,j,1]  #jth check node connected to variable node i
                #jj=find(CtoV[cn ,:,1].==i)[1]
                jj=LH.VtoC[i,j,2]
                
                mn[i]+=mCtoV[cn,jj]
            end
            if mn[i] <= 0.0
                bn[i]=1
            else
                bn[i]=0
            end
        end
        
        
        if isCW(LH,bn)==1
            return bn
        end
    end
    return bn
end

bsc_MPD (generic function with 1 method)

# Here we generate an LDPC code

In [13]:
#set channel /code parameters
n=1000
dv=3
dc=6

m=int64(n*dv/dc)
k=n-m

LH=ldpcH
mg=iceil(2*ell(dv,dc,n))
println("Code has rate $(k/n) ")
println("($dv,$dc)-code\nn=$n m=$m \nMust have girth greater then $(2*ell(dv,dc,n))")
LH = ldpcH(ones(Int64,n,1)*dv,ones(Int64,m,1)*dc);
if LH==false
    println("Try again, we were unable to randomly generate a LDPC code with the desired parameters")
else
    H=calc_H(LH)
    G=calc_G(H);
    if G!= false
        println("We are succesful!")
    else
        println("Try again.")
    end
end



Code has rate 0.5 
(3,6)-code
n=1000 m=500 
Must have girth greater then 6.249877473216599
We are succesful!


# Here we generate a random message, pass it through the erasure channel, and then use message passing to decode

In [14]:
sk=rand(Bernoulli(0.5),k)   #source message
maxi=1000 #maximum iterations for message passing
d=0.3 #erasure probability

xn=mod(G'*sk,2)
yn= bec(d,xn) #erasures are denoted by 2

cnt=0 
for i=1:n #count the # of erasures
    if yn[i]==2
        cnt+=1
    end
end
println("The # of erasures is $cnt")

xnp=bec_MPD(yn,LH,maxi) #run message passing for maxi iterations
 ne=0
rgt=0
for i=1:n 
    if xnp[i]==2
        ne+=1 
    end
    if xnp[i] == xn[i]
        rgt+=1
    end
end
println("# of erasures after MPD $ne")
println(" n=$n #right= $rgt  BER= $(1.0-rgt/n)")

The # of erasures is 299
# of erasures after MPD 0
 n=1000 #right= 1000  BER= 0.0


# Here we generate a random message, pass it through the binary symmetric channel, and then use message passing to decode

In [15]:
sk=rand(Bernoulli(0.5),k)   #source message
maxi=1000 #maximum iterations for message passing
d=0.08 #bsc probability

xn=mod(G'*sk,2)
yn= bsc(d,xn) 

cnt=0 
for i=1:n #count the # of flips
    if yn[i] != xn[i]
        cnt+=1
    end
end
println("The # of bit flips is $cnt")

xnp=bsc_MPD(yn,d,LH,maxi) #run message passing for maxi iterations
             
ne=0
rgt=0
for i=1:n 
    if xnp[i]!=xn[i]
        ne+=1 
    else
        rgt+=1
    end
end
println("# of flips after MPD $ne")
println(" n=$n #right= $rgt  BER= $(1.0-rgt/n)")

The # of bit flips is 72
# of flips after MPD 0
 n=1000 #right= 1000  BER= 0.0
